In [4]:
import pandas as pd
import os

"""
Como ira organizado el DataFrame

Games | ReleaseDate | CharactersPath | Developer | 2D | 3D | TagTeam | AirDashers | Anime | WeaponBased | Fast-paced | Footsies | SlowPaced | Installbased | CrossOver
Nombre | Fecha de Lanzamiento | Ruta_en_dir_data (del archivo con los links de personajes)|ImageLink | Desarrollador | y el resto con una X mayuscula si tiene esa caracteristica o un espacio si no la tiene
"""


df = pd.DataFrame({"Games":[],
                   "ReleaseDate":[],
                   "CharactersPath":[],
                   "ImageLink":[], #Link de la imagen del juego 
                   "2D":[], 
                   "3D":[], 
                   "TagTeam":[], 
                   "AirDashers":[], 
                   "Anime":[], 
                   "WeaponBased":[], 
                   "Fast-paced":[],
                   "Footsies":[],
                   "SlowPaced":[],
                   "Installbased":[],
                   "CrossOver":[]
                   })



#funcion que crea un archivo csv con el nombre del juego y la link de discord de la imagen de los personajes.

def CreateCharacterFile(GameName)-> None:
    file_name =  f"Data/{GameName}.csv"

    if os.path.exists(file_name):
        
        return None

    with open(file_name, "w") as f:

        # First row is the header, put CharacterName and ImageLink
        f.write("CharacterName,ImageLink,Archetype,Archetype2,Archetype3,Archetype4,Archetype5\n")
        f.close()



In [5]:

"""
Agregar un nuevo juego por medio de los archivos tabulados
Los archivos tabulados deben de estar en la carpeta TabulatedData
"""

with open(f"TabulatedData/GameName.csv", "r") as f:
    rows_to_add = []
    for line in f:
        # Desempaquetar y eliminar espacios en blanco
        parts = list(map(str.strip, line.split(",")))
        if len(parts) == 2:
            GameName, ReleaseDate = parts
            # Diccionario para la nueva fila
            new_row = {
                "Games": GameName,
                "ReleaseDate": ReleaseDate,
                "CharactersPath": f"Data/{GameName}.csv",
                "ImageLink": "",
                "2D": "",
                "3D": "",
                "TagTeam": "",
                "AirDashers": "",
                "Anime": "",
                "WeaponBased": "",
                "Fast-paced": "",
                "Footsies": "",
                "SlowPaced": "",
                "Installbased": "",
                "CrossOver": ""
            }
            # Añadir a la lista de filas a agregar
            CreateCharacterFile(GameName)
            rows_to_add.append(new_row)
        else:
            print(f"Línea ignorada debido a formato incorrecto: {line}")

    # Usar pandas.concat para agregar todas las filas al DataFrame
    df = pd.concat([df, pd.DataFrame(rows_to_add)], ignore_index=True)  

Línea ignorada debido a formato incorrecto: 

Línea ignorada debido a formato incorrecto: 

Línea ignorada debido a formato incorrecto: 

Línea ignorada debido a formato incorrecto: 



In [24]:
import hashlib
import time

def hasher_function(To_Hash):
    hasher = hashlib.sha256()
    
    for hash in To_Hash:
        hasher.update(To_Hash.encode('utf-8'))
    

    current_time = str(int(time.time()))  
    hasher.update(current_time.encode('utf-8'))
    

    hash_value = hasher.hexdigest()
    
    return "U" + hash_value


In [7]:
import pandas as pd
import neo4j as neo
import os
from dotenv import load_dotenv

load_dotenv()

# Driver for graph
driver = neo.GraphDatabase.driver(os.getenv('NEO4J_URI'), auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD')))

# Leer el CSV
csv_path = "NewGames.csv"
data = pd.read_csv(csv_path)


# ** Run this query when all the data files are filled
# Crear nodos y relaciones en Neo4j
with driver.session() as session:
    # Crear nodos para juegos
    for index, row in data.iterrows():
        try:
            game_name = row['Games']
            release_date = row['ReleaseDate']
            characters_path = row['CharactersPath']
            image_link = row['ImageLink']

            session.run("CREATE (g:Reconode {name: $name, image_link: $image_link})",
                        name=game_name, image_link=image_link)

            characters_path = f"Data/{game_name}.csv"
            dfs = pd.read_csv(characters_path)

            for index,rowcharacter in dfs.iterrows():
                character_name = rowcharacter['CharacterName']
                print(character_name)
                SearchLink = "https://www.google.com/search?q=" + character_name + " " + game_name + " character" + "SuperWikiCombo"

                session.run("CREATE (c:Character {name: $name, game: $game_name, search_link: $search_link})",
                            name=character_name, game_name=game_name, search_link=SearchLink)

                session.run("MATCH (g:Reconode {name: $game_name}), (c:Character {name: $character_name}) CREATE (c)-[:From]->(g)",
                            game_name=game_name, character_name=character_name)         
            
            #Para cada categoría, si hay una 'X', crear el nodo de categoría y la relación
            category_columns = ["2D", "3D", "TagTeam", "AirDashers", "Anime", "WeaponBased", "Fast-paced", 
                                "Footsies", "SlowPaced", "Installbased", "CrossOver"]

            for category in category_columns:
                if pd.notna(row[category]) and (row[category] == 'X' or row[category] == 'x'):
                    print(category)
                    session.run("MERGE (c:Label {name: $category})", category=category)

                    session.run("MATCH (g:Reconode {name: $game_name}), (c:Label {name: $category}) CREATE (g)-[:BELONGS_TO]->(c)",
                                game_name=game_name, category=category)
        except Exception as e:
            pass
        

driver.close()



Kazuya Mishima
Paul Phoenix
Marshall Law
Jack
Michelle Chang
King
Nina Williams
Yoshimitsu
Prototype Jack
3D
SlowPaced
Heihachi Mishima
Kazuya Mishima
Lee Chaolan
Lei Wulong
Nina Williams
Paul Phoenix
Yoshimitsu
Jack-2
King
Michelle Chang
Armor King
Ganryu
Kunimitsu
Baek Doo San
Bruce Irvin
Anna Williams
Wang Jinrei
Devil
Angel
3D
SlowPaced
Jin Kazama
Anna Williams
Armor King II
Bryan Fury
Eddy Gordo
Forest Law
Gon
Gun Jack
Hwoarang
Julia Chang
King II
Kuma II
Lei Wulong
Ling Xiaoyu
Nina Williams
Paul Phoenix
Tiger Jackson
True Ogre
Yoshimitsu
3D
SlowPaced
Jin Kazama
Anna Williams
Armor King II
Bryan Fury
Eddy Gordo
Forest Law
Gon
Gun Jack
Hwoarang
Julia Chang
King II
Kuma II
Lei Wulong
Ling Xiaoyu
Nina Williams
Paul Phoenix
Tiger Jackson
True Ogre
Yoshimitsu
3D
Footsies
Anna Williams
Asuka Kazama
Baek Doo San
Bruce Irvin
Bryan Fury
Christie Monteiro
Craig Marduk
Devil Jin
Eddy Gordo
Feng Wei
Ganryu
Heihachi Mishima
Hwoarang
Jack-5
Jin Kazama
Julia Chang
Kazuya Mishima
King
Kuma
Lee Ch

In [25]:
import pandas as pd
import neo4j as neo
import os
%pip install python-dotenv
from dotenv import load_dotenv

load_dotenv()
driver = neo.GraphDatabase.driver(os.getenv('NEO4J_URI'), auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD')))

usuarios_csv_path = "Respuestas.csv"
usuarios_data = pd.read_csv(usuarios_csv_path)
with driver.session() as session:
        for index, row in usuarios_data.iterrows():
            # Obtener el nombre del usuario segun el CSV
            user_name = row['Usuarios']
            hashed_user_name = hasher_function(user_name)  # Hashear el nombre de usuario
            preferences = row.drop('Usuarios').to_dict()
            print(preferences)
            # Crear nodo de usuario con el nombre hasheado
            session.run("CREATE (u:User {name: $name})", name=hashed_user_name)
            
            # Vincular el usuario a los juegos según sus preferencias
            for game, value in preferences.items():
                if pd.notna(value) and (value == 'x'):
                    # Vincular el usuario al "Reconode" del juego
                    session.run("MATCH (u:User {name: $user_name}), (g:Reconode {name: $game_name}) CREATE (u)-[:LIKES]->(g)",
                                user_name=hashed_user_name, game_name=game)

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
{'2D': 'x', '3D': 'x', 'TagTeam': 'x', 'AirDashers': nan, 'Anime': nan, 'WeaponBased': nan, 'Fast-paced': nan, 'Footsies': nan, 'SlowPaced': nan, 'Installbased': nan, 'CrossOver': nan}
{'2D': nan, '3D': nan, 'TagTeam': 'x', 'AirDashers': nan, 'Anime': 'x', 'WeaponBased': nan, 'Fast-paced': 'x', 'Footsies': nan, 'SlowPaced': 'x', 'Installbased': 'x', 'CrossOver': nan}
{'2D': nan, '3D': nan, 'TagTeam': nan, 'AirDashers': nan, 'Anime': nan, 'WeaponBased': 'x', 'Fast-paced': nan, 'Footsies': nan, 'SlowPaced': 'x', 'Installbased': nan, 'CrossOver': 'x'}
{'2D': nan, '3D': nan, 'TagTeam': nan, 'AirDashers': nan, 'Anime': nan, 'WeaponBased': nan, 'Fast-paced': nan, 'Footsies': nan, 'SlowPaced': 'x', 'Installbased': 'x', 'CrossOver': 'x'}
{'2D': nan, '3D': nan, 'TagTeam': nan, 'AirDashers': 'x', 'Anime': nan, 'WeaponBased': nan, 'Fast-paced': nan, 'Foot

In [1]:
import pandas as pd
import neo4j as neo
import os
from dotenv import load_dotenv

load_dotenv()
driver = neo.GraphDatabase.driver(os.getenv('NEO4J_URI'), auth=(os.getenv('NEO4J_USERNAME'), os.getenv('NEO4J_PASSWORD')))


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
